In [34]:
# BeautifulSoup
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import csv
from csv import writer

# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
driver.maximize_window()

movie_name = []
genre = []
certificate=[]
year = []
runtime = []
rating = []
metascore = []
votes = []
gross = []
description = []
director = []
stars = []


driver.get("https://www.imdb.com/search/title/?title_type=feature")


# time.sleep(3)
# for i in range(1,2):
#     report1 = driver.find_elements(By.CLASS_NAME,"ipc-btn.ipc-btn--single-padding.ipc-btn--center-align-content.ipc-btn--default-height.ipc-btn--core-base.ipc-btn--theme-base.ipc-btn--on-accent2.ipc-text-button.ipc-see-more__button")[0]
#     driver.execute_script("arguments[0].click();", report1)
#     time.sleep(5)
    
# movie_data = driver.find_elements(By.CLASS_NAME,"ipc-icon-button.dli-info-icon.ipc-icon-button--base.ipc-icon-button--onAccent2")
# ipc-title-link-wrapper
movie_data = driver.find_elements(By.CLASS_NAME,"sc-b4e41383-4.efDLHx.dli-parent")


In [44]:
headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36"}
error = []
for store in movie_data[:5]:
#     try:
    store_html = store.get_attribute("outerHTML")
    soup = BeautifulSoup(store_html, 'html.parser')
    name = soup.find('h3',class_="ipc-title__text").text.split(" ")[1]
    duration = soup.find('div',class_="sc-1e00898e-7 hcJWUf dli-title-metadata").text
    certificate= duration.split("m")[1]
    time = duration.split("m")[0]
    watch_time = time.split("h")[0][-1]+ "h"+time.split("h")[1]+"m"
    year = time.split("h")[0][:-1]
    metascore_rate = soup.find('span', class_="sc-1e00898e-1 dGyusp").text.split("Metascore")[0].split("Rate")
    metascore= metascore_rate[1]
    rate = metascore_rate[0].split('\xa0')[0]
    description = soup.find('div', class_="ipc-html-content-inner-div").text
    votes = soup.find('div', class_="sc-b4e41383-0 cyGaqI").text.split("Votes")[1]


    url_name = store.find_elements(By.CLASS_NAME,"ipc-title-link-wrapper")[0]
    url = f"{url_name.get_attribute('href')}"
#     request allow you to send HTTP request
    response = requests.get(url,headers=headers)
    soup2 = BeautifulSoup(response.content, 'html.parser')
    gen = soup2.find('div', class_="ipc-chip-list__scroller")
    gen2 = gen.findAll('a', class_="ipc-chip ipc-chip--on-baseAlt")

    genre = [a.text for a in gen2]

    dri_wri_star_ul= soup2.find('ul', class_='ipc-metadata-list ipc-metadata-list--dividers-all title-pc-list ipc-metadata-list--baseAlt')
    dri_wri_star_li = dri_wri_star_ul.findAll('li', class_='ipc-metadata-list__item')
    dri_wri_star = dri_wri_star_ul.find('li', class_='ipc-metadata-list__item ipc-metadata-list-item--link')
    writer = []
    director = []
    for li in dri_wri_star_li:
        if li.span and li.span.text in ['Writer', 'Writers']:
            wri = li.findAll('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
            writer+= [a.text for a in wri]
        if li.span and li.span.text in ['Director', 'Directors']:
            dri = li.findAll('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
            director +=[a.text for a in dri]

    sta = li.findAll('a', class_='ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link')
    star = [a.text for a in sta]
    gross_data =  soup2.findAll('li', class_="ipc-metadata-list__item sc-48038814-2 MkXSe")

    bud=None
    gros=None

    for li in gross_data:

        if li.span and li.span.text=="Budget":
            bud=li.find('span', class_='ipc-metadata-list-item__list-content-item').text
        if li.span and li.span.text=="Gross worldwide":
            gros=li.find('span', class_='ipc-metadata-list-item__list-content-item').text   

    movie_DF = {
        'Name': name,
        'Year': year,
        'Genre': ",".join(genre) if len(genre) >0 else None,
        'Watchtime': watch_time,
        'Certificate': certificate,
        'Rating':rate,
        'Mtascore': metascore,
        'Director': ",".join(director) if len(director) >0 else None,
        'Writer': ",".join(writer) if len(writer) >0 else None,
        'Stars': ",".join(star) if len(star) >0 else None,
        'Votes': votes,
        'Budget': bud,
        'Gross': gros,
        'Description': description,
    }

    with open('mycsvfile.csv', 'w') as f:  # You will need 'wb' mode in Python 2.x
        w = csv.DictWriter(f, movie_DF.keys())
        w.writeheader()
        w.writerow(movie_DF)
        
#     except:
#         movie_name=store.find_elements(By.CLASS_NAME,"ipc-title__text")[0].text
#         error.append(movie_name)
#         pass
print(len(error), error)

0 []


IndentationError: unexpected indent (2361666311.py, line 8)